# VBA Files

### Pulling Excel Files from the Website

In [1]:
# There are weekly files from 2014 to present on https://www.benefits.va.gov/reports/detailed_claims_data.asp

# There are historical files at https://www.benefits.va.gov/REPORTS/mmwr/historical/{year}/index.asp from 2004 to 2013

# Presumably the first step will be just pulling all of them

### Libraries/Setup

In [2]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
os.chdir(r'C:\Users\admin\Documents\Python Scripts\veteranLawData')

### Get File URLs

In [3]:
def pullTheFiles(year):
    url= f'https://www.benefits.va.gov/REPORTS/mmwr/historical/{str(year)}/index.asp'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    filesToGet=[]
    for link in soup.findAll('a'):
        try:
            if "xls" in (link.get('href')):
                filesToGet.append(link.get('href'))
        except:
            pass
    return(filesToGet)

yearRange=list(range(2004, 2014))
#files=pullTheFiles(2004)
filesList=list(map(pullTheFiles, yearRange))
flatFileList = [item for sublist in filesList for item in sublist]

### Check We Have The Right Number: 2008 Is Legitimately Missing Three, I Checked

In [4]:
for fileList in filesList:
    print(len(fileList))

52
52
52
52
49
52
52
52
52
53


### Pull The Files

In [5]:
def pullFile(url):
    cleanLink='https://www.benefits.va.gov'+url
    r = requests.get(cleanLink, allow_redirects=True)
    name=cleanLink.split("/")[-1]
    open(name, 'wb').write(r.content)
    return(name)

#listOfFileNames=list(map(pullFile, files))
filesNameList=list(map(pullFile, flatFileList))

### Read Them Into A Dictionary

In [8]:
def readBasicFile(fileName):
    try:
        df=pd.read_excel(fileName)
        return(df)
    except:
        print(f"failed on {fileName}")

listOfDFs=list(map(readBasicFile, filesNameList))
dictionaryOfFiles = dict(zip(filesNameList, listOfDFs))

failed on 102113.xls
failed on 102813.xls
failed on 110413.xls
failed on 111213.xls
failed on 111813.xls
failed on 112513.xls
failed on 120213.xls
failed on 120913.xls
failed on 121613.xls
failed on 122313.xls
failed on 123013.xls


### Append Into Master DF

In [11]:
masterDF=pd.DataFrame()
for key in dictionaryOfFiles.keys():
    try:
        df=dictionaryOfFiles[key]
        df['fileName']=key
        listOfPhrasesToFind=["As of", "For the Week"]
        foundPhrase=[i for i in df.columns if any(substring in i for substring in listOfPhrasesToFind)]
        dropPhrase=[i for i in df.columns if "Percent" in i]
        #print(len(foundPhrase))
        df["forTheWeek"]=df[foundPhrase[0]]
        df=df.drop([foundPhrase[0]], axis=1)
        df=df.drop(dropPhrase, axis=1)
        masterDF=masterDF.append(df, sort=True)
    except:
        print(f'failed on {key}')

failed on NewMMWL_100509.xls
failed on 101309.xls
failed on 101909.xls
failed on 102609.xls
failed on 110209.xls
failed on 110909.xls
failed on 111609.xls
failed on 112309.xls
failed on 113009.xls
failed on 120709.xls
failed on 121409.xls
failed on 122109.xls
failed on 122809.xls
failed on 010410.xlsx
failed on 040510.xlsx
failed on 070610.xlsx
failed on 100410.xlsx
failed on 011110.xlsx
failed on 041210.xlsx
failed on 071210.xlsx
failed on 101210.xlsx
failed on 011910.xlsx
failed on 041910.xlsx
failed on 071910.xlsx
failed on 101810.xlsx
failed on 012510.xlsx
failed on 042610.xlsx
failed on 072610.xlsx
failed on 102510.xlsx
failed on 020110.xlsx
failed on 050310.xlsx
failed on 080210.xlsx
failed on 110110.xlsx
failed on 020810.xlsx
failed on 051010.xlsx
failed on 080910.xlsx
failed on 110810.xlsx
failed on 021610.xlsx
failed on 051710.xlsx
failed on 081610.xlsx
failed on 111510.xlsx
failed on 022210.xlsx
failed on 052410.xlsx
failed on 082310.xlsx
failed on 112210.xlsx
failed on 03011

In [12]:
masterDF.columns

Index([' ', ' Rating Cases Pending', ' Rating Claims Pending',
       '* Scorecard  Non-Rating Cases Pending',
       '* Scorecard  Non-Rating Claims Pending', 'C&P Claims over 180 Days ',
       'C&P Claims over 180 Days in WIPP', 'C&P Work Items over 180 Days ',
       'COE', 'COEs Issued', 'Date of Oldest Pending COE',
       'Education Cases Pending ', 'Education Work Items Pending ',
       'Guarantees Pending', 'IVMs Pending (EP 154 and EP 314)',
       'Non-Rating Cases Pending over 180 Days',
       'Non-Rating Claims Pending over 180 Days', 'Pre-Discharge Claims',
       'Rating Cases Pending', 'Rating Cases Pending over 180 Days',
       'Rating Claims Pending over 180 Days', 'SOC's',
       'Scorecard Non-Rating Cases Pending', 'Scorecard Rating Cases Pending',
       'Total Appeals Requiring Adjudicative Action ',
       'Total Appeals Requiring Adjudicative action ', 'Total C&P Pending ',
       'Total C&P Pending in Wipp', 'Total C&P Work Items Pending ',
       'Unnamed:

### Right now: it looks like 04-09 is working; after that we need to look at the format of the 09 worksheets to modify it; there's also some column cleaning we may need to do for the 04-09 group; this is a good stopping point